In [18]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import udf

sc = pyspark.SparkContext()
sc.addPyFile('/home/hadoop/graphframes-0.7.0-spark2.4-s_2.11.jar')
spark = SparkSession.builder.appName("SparkSession").getOrCreate()

In [19]:
from graphframes import *
import hashlib

In [39]:
df = spark.read.load("test/",
                     format="csv", sep="\t", inferSchema="true", header="false")

In [37]:
GraphFrame

graphframes.graphframe.GraphFrame

In [21]:
df.show(10)

+--------------------+--------------------+
|                 _c0|                 _c1|
+--------------------+--------------------+
|           002224.cn|       hd.hstong.com|
|           002366.cn|       hd.hstong.com|
|           002366.cn|       hd.hstong.com|
|01278.outlookrefl...|           nginx.org|
|            0311.top|cloudcdn.dopa.com.cn|
|            0311.top|cloudcdn.dopa.com.cn|
|            0311.top|ms.ok365.com?dm=0...|
|            0311.top|cloudcdn.dopa.com.cn|
|            0311.top|cloudcdn.dopa.com.cn|
|            0311.top|cloudcdn.dopa.com.cn|
+--------------------+--------------------+
only showing top 10 rows



In [8]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [26]:
df.count()

320044

In [30]:
df = df.withColumnRenamed('_c0', 'parentTLD').withColumnRenamed('_c1', 'childTLD').filter("parentTLD is not null and childTLD is not null")

In [31]:
df = df.drop_duplicates()
df.show(5)

+--------------------+-------------+
|           parentTLD|     childTLD|
+--------------------+-------------+
|     24hourebook.com|  jlsl.gov.cn|
|         3arbyat.com| fmprc.gov.cn|
|674.yunshenggongy...|1.ssbet081.cn|
|   7bbaq.hrbzyzs.com|h.ssbet005.cn|
|   7bbaq.hrbzyzs.com|b.ssbet219.cn|
+--------------------+-------------+
only showing top 5 rows



In [32]:
aggcodes = df.select("parentTLD","childTLD").rdd.flatMap(lambda x: x).distinct()
aggcodes.count()

84248

In [33]:
def hashnode(x):
    return hashlib.sha1(x.encode("UTF-8")).hexdigest()[:8]

hashnode_udf = udf(hashnode)

In [34]:
vertices = aggcodes.map(lambda x: (hashnode(x), x)).toDF(["id","name"])

vertices.show(5)

+--------+--------------------+
|      id|                name|
+--------+--------------------+
|1ba1e00f|       h.ssbet005.cn|
|465806fb|         twitter.com|
|0d5f83d9|biblioteca.cultur...|
|2f7d968c|     blog.sitetag.us|
|50613cfc|     set.jjfsfjjg.cn|
+--------+--------------------+
only showing top 5 rows



In [35]:
edges = df.select("parentTLD","childTLD")\
.withColumn("src", hashnode_udf("parentTLD"))\
.withColumn("dst", hashnode_udf("childTLD"))\
.select("src","dst")

edges.show(5)

+--------+--------+
|     src|     dst|
+--------+--------+
|a955fc5f|ec156376|
|c4bf9b94|11194bcc|
|4a821622|b490fbac|
|e09af9ee|1ba1e00f|
|e09af9ee|f4859c63|
+--------+--------+
only showing top 5 rows



In [38]:
graph = GraphFrame(vertices, edges)

Py4JJavaError: An error occurred while calling o458.loadClass.
: java.lang.ClassNotFoundException: org.graphframes.GraphFramePythonAPI
	at java.net.URLClassLoader.findClass(URLClassLoader.java:382)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:419)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:352)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thr

In [16]:
spark.stop()